# Data Quality Audit

Comprehensive analysis of feature engineering pipeline data quality:
- Data types and memory efficiency
- Extreme outliers and impossible values
- Near-zero division issues
- Categorical feature handling
- Missing value patterns

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

PROJ_ROOT = Path.cwd().parent
if str(PROJ_ROOT) not in sys.path:
    sys.path.append(str(PROJ_ROOT))

from credit_risk_xai.config import FEATURE_CACHE_PATH, FEATURES_FOR_MODEL, CATEGORICAL_COLS
from credit_risk_xai.features.engineer import prepare_modeling_data
from credit_risk_xai.plotting import set_thesis_style, FIGSIZE, COLORS, save_figure, despine

# Initialize thesis-quality plotting style
set_thesis_style(use_tex=True)

FIGURES_DIR = PROJ_ROOT / "figures"

%matplotlib inline

## 1. Load Data and Basic Overview

In [ ]:
# Load feature-engineered dataset
df = pd.read_parquet(FEATURE_CACHE_PATH)

# Apply standard filters
df_filtered = df[
    (df['ser_aktiv'] == 1) & 
    (df['sme_category'].isin(['Small', 'Medium'])) & 
    (df['knc_kncfall'] == 1) &
    (df["bransch_borsbransch_konv"] != "40.0")
]

print(f"Total records: {len(df):,}")
print(f"After filters: {len(df_filtered):,}")
print(f"\nMemory usage:")
print(f"  Full dataset: {df.memory_usage(deep=True).sum() / 1024**3:.2f} GB")
print(f"  Filtered: {df_filtered.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Get modeling features
X, y = prepare_modeling_data(df_filtered)
print(f"\nModeling data: {len(X):,} rows × {len(X.columns)} features")
print(f"Target distribution: {y.value_counts().to_dict()}")

In [ ]:
df_duplicates = df_filtered[
    (df_filtered['target'] == 1) &
    (df_filtered['ny_omsf'] == 0.0)
    ]
df_duplicates.head(20)

## 2. Data Type Analysis

In [ ]:
print("=" * 100)
print("DATA TYPE ANALYSIS")
print("=" * 100)

# Analyze dtypes
dtype_summary = X.dtypes.value_counts()
print("\nData type distribution:")
print(dtype_summary)

# Check categorical dtypes
print("\n" + "-" * 100)
print("CATEGORICAL FEATURES ANALYSIS")
print("-" * 100)

categorical_status = []
for col in CATEGORICAL_COLS:
    if col in X.columns:
        is_category = X[col].dtype.name == 'category'
        n_unique = X[col].nunique()
        categorical_status.append({
            'feature': col,
            'current_dtype': str(X[col].dtype),
            'is_category': '✓' if is_category else '✗',
            'unique_values': n_unique,
            'null_pct': f"{100 * X[col].isna().sum() / len(X):.1f}%"
        })

cat_df = pd.DataFrame(categorical_status)
print(cat_df.to_string(index=False))

# Dtype efficiency check
print("\n" + "-" * 100)
print("DTYPE EFFICIENCY ISSUES")
print("-" * 100)

for col in X.columns:
    dtype = X[col].dtype
    
    # Check if using oversized int
    if dtype == 'Int16' or dtype == 'int16':
        min_val, max_val = X[col].min(), X[col].max()
        if min_val >= 0 and max_val <= 255:
            print(f"  {col}: Using {dtype}, but range [{min_val}, {max_val}] fits in uint8")
    
    # Check if using float64 unnecessarily
    if dtype == 'float64':
        print(f"  {col}: Using float64 (could use float32 if precision not critical)")

## 3. Feature Distribution Analysis

In [ ]:
print("=" * 100)
print("DESCRIPTIVE STATISTICS (All Features)")
print("=" * 100)

# Get numeric columns only
numeric_cols = X.select_dtypes(include=[np.number]).columns
desc = X[numeric_cols].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T

# Add null percentage
desc['null_pct'] = 100 * X[numeric_cols].isna().sum() / len(X)

# Add outlier count (beyond 5 std devs)
outlier_counts = []
for col in numeric_cols:
    values = X[col].dropna()
    if len(values) > 0:
        mean, std = values.mean(), values.std()
        if std > 0:
            outliers = (np.abs(values - mean) > 5 * std).sum()
        else:
            outliers = 0
    else:
        outliers = 0
    outlier_counts.append(outliers)

desc['outliers_5std'] = outlier_counts

print(desc.to_string())

## 4. Near-Zero Division Issues

Identify features with extreme values likely caused by division by near-zero denominators.

In [ ]:
print("=" * 100)
print("NEAR-ZERO DIVISION ANALYSIS")
print("=" * 100)

# Features known to have division operations
division_features = {
    'dso_days': {'numerator': 'br06g_kfordsu', 'denominator': 'rr01_ntoms', 'reasonable_range': (0, 365)},
    'dpo_days': {'numerator': 'br13a_ksklev', 'denominator': 'rr06a_prodkos', 'reasonable_range': (0, 365)},
    'ratio_cash_liquidity': {'numerator': 'br08_omstgsu', 'denominator': 'br13_ksksu', 'reasonable_range': (0, 10)},
    'ratio_cash_interest_cov': {'numerator': None, 'denominator': None, 'reasonable_range': (-100, 100)},
    'rr07_rorresul_yoy_pct': {'numerator': None, 'denominator': 'prev_year_value', 'reasonable_range': (-100, 100)},
    'ratio_cash_liquidity_yoy_pct': {'numerator': None, 'denominator': 'prev_year_value', 'reasonable_range': (-100, 100)},
    'current_ratio_yoy_pct': {'numerator': None, 'denominator': 'prev_year_value', 'reasonable_range': (-100, 100)},
}

for feature, info in division_features.items():
    if feature not in X.columns:
        continue
    
    print(f"\n{feature}:")
    print("-" * 80)
    
    values = X[feature].dropna()
    
    # Basic stats
    print(f"  Count: {len(values):,} | Null: {X[feature].isna().sum():,} ({100*X[feature].isna().sum()/len(X):.1f}%)")
    print(f"  Min: {values.min():.2f} | Max: {values.max():.2f}")
    print(f"  Mean: {values.mean():.2f} | Median: {values.median():.2f}")
    print(f"  P1: {values.quantile(0.01):.2f} | P99: {values.quantile(0.99):.2f}")
    
    # Check reasonable range
    reasonable_min, reasonable_max = info['reasonable_range']
    in_range = (values >= reasonable_min) & (values <= reasonable_max)
    pct_in_range = 100 * in_range.sum() / len(values)
    
    print(f"  In reasonable range [{reasonable_min}, {reasonable_max}]: {pct_in_range:.1f}%")
    print(f"  Outside reasonable range: {(~in_range).sum():,} obs ({100*(~in_range).sum()/len(values):.1f}%)")
    
    # Extreme values
    extreme_low = values < (reasonable_min - 1000)
    extreme_high = values > (reasonable_max + 1000)
    
    if extreme_low.sum() > 0:
        print(f"  ⚠ EXTREME LOW (<{reasonable_min - 1000}): {extreme_low.sum():,} obs")
        print(f"    Worst 5: {values[extreme_low].nsmallest(5).values}")
    
    if extreme_high.sum() > 0:
        print(f"  ⚠ EXTREME HIGH (>{reasonable_max + 1000}): {extreme_high.sum():,} obs")
        print(f"    Worst 5: {values[extreme_high].nlargest(5).values}")

## 5. Denominator Analysis

Check the distributions of denominators to understand near-zero division issues.

In [ ]:
print("=" * 100)
print("DENOMINATOR ANALYSIS (for division-based features)")
print("=" * 100)

# Check key denominators
denominators = {
    'rr01_ntoms': 'Revenue (used in DSO calculation)',
    'rr06a_prodkos': 'Production costs (used in DPO calculation)',
    'br13_ksksu': 'Current liabilities (used in cash liquidity ratio)',
}

for denom_col, description in denominators.items():
    if denom_col not in df_filtered.columns:
        continue
    
    print(f"\n{denom_col}: {description}")
    print("-" * 80)
    
    values = df_filtered[denom_col].dropna()
    
    # Basic stats
    print(f"  Count: {len(values):,}")
    print(f"  Min: {values.min():.2f} | Max: {values.max():.2f}")
    print(f"  Mean: {values.mean():.2f} | Median: {values.median():.2f}")
    
    # Check for near-zero and negative
    zero_count = (values == 0).sum()
    near_zero_1 = (values.abs() < 1).sum()
    near_zero_10 = (values.abs() < 10).sum()
    negative = (values < 0).sum()
    
    print(f"\n  Zero: {zero_count:,} ({100*zero_count/len(values):.2f}%)")
    print(f"  |value| < 1: {near_zero_1:,} ({100*near_zero_1/len(values):.2f}%)")
    print(f"  |value| < 10: {near_zero_10:,} ({100*near_zero_10/len(values):.2f}%)")
    print(f"  Negative: {negative:,} ({100*negative/len(values):.2f}%)")
    
    # Recommended threshold
    if 'ntoms' in denom_col:  # Revenue
        threshold = 10.0
    else:
        threshold = 1.0
    
    below_threshold = (values.abs() < threshold).sum()
    print(f"\n  ⚠ WOULD EXCLUDE with threshold={threshold}: {below_threshold:,} ({100*below_threshold/len(values):.1f}%)")

## 6. Visualization: Extreme Value Distributions

In [ ]:
# Plot distributions of problematic features
problematic_features = [
    'dso_days', 'dpo_days', 'ratio_cash_liquidity', 
    'rr07_rorresul_yoy_pct', 'ratio_cash_liquidity_yoy_pct'
]

fig, axes = plt.subplots(2, 3, figsize=(FIGSIZE['standalone'][0], 3.5))
axes = axes.flatten()

for idx, feature in enumerate(problematic_features):
    if feature not in X.columns or idx >= len(axes):
        continue
    
    ax = axes[idx]
    values = X[feature].dropna()
    
    # Use percentile range for visualization
    p1, p99 = values.quantile([0.01, 0.99])
    filtered_values = values[(values >= p1) & (values <= p99)]
    
    # Histogram
    ax.hist(filtered_values, bins=50, alpha=0.7, edgecolor='white', linewidth=0.3,
            color=COLORS['lgbm'])
    ax.set_title(feature.replace('_', r'\_'), fontsize=8)
    ax.set_xlabel('Value', fontsize=7)
    ax.set_ylabel('Frequency', fontsize=7)
    ax.tick_params(labelsize=6)
    
    # Add stats text
    stats_text = f"P1--P99: [{p1:.1f}, {p99:.1f}]\nOutside: {(~values.between(p1, p99)).sum():,}"
    ax.text(0.98, 0.98, stats_text, transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='#cccccc'),
            fontsize=5)
    
    despine(ax)

# Hide extra subplot
if len(problematic_features) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
save_figure(fig, FIGURES_DIR / 'feature_distributions_audit.pdf')
plt.show()

## 7. Summary and Recommendations

In [ ]:
print("="*100)
print("SUMMARY: REQUIRED FIXES")
print("="*100)

print("\n1. CATEGORICAL DTYPES")
print("-" * 80)
cat_not_encoded = [col for col in CATEGORICAL_COLS if col in X.columns and X[col].dtype.name != 'category']
if cat_not_encoded:
    print(f"  ✗ {len(cat_not_encoded)} categorical features not using category dtype:")
    for col in cat_not_encoded:
        print(f"    - {col}: {X[col].dtype}")
else:
    print("  ✓ All categorical features properly encoded")

print("\n2. NEAR-ZERO DIVISION ISSUES")
print("-" * 80)
print("  Recommend implementing threshold-based NaN conversion in _safe_div():")
print("    - min_abs_denom=1.0 for most ratios (1k SEK)")
print("    - min_abs_denom=10.0 for revenue-based metrics (10k SEK)")
print("\n  Expected impact:")

# Estimate impact for key denominators
if 'rr01_ntoms' in df_filtered.columns:
    revenue_below_10 = (df_filtered['rr01_ntoms'].abs() < 10).sum()
    print(f"    - DSO: ~{100*revenue_below_10/len(df_filtered):.1f}% → NaN (revenue < 10k)")

if 'rr06a_prodkos' in df_filtered.columns:
    prodcost_below_1 = (df_filtered['rr06a_prodkos'].abs() < 1).sum()
    print(f"    - DPO: ~{100*prodcost_below_1/len(df_filtered):.1f}% → NaN (prod costs < 1k)")

if 'br13_ksksu' in df_filtered.columns:
    liab_below_1 = (df_filtered['br13_ksksu'].abs() < 1).sum()
    print(f"    - Cash liquidity ratio: ~{100*liab_below_1/len(df_filtered):.1f}% → NaN (liabilities < 1k)")

print("\n3. DATA TYPE OPTIMIZATION")
print("-" * 80)
if 'event_count_last_5y' in X.columns:
    print(f"  • event_count_last_5y: {X['event_count_last_5y'].dtype} → uint8 (range 0-5)")
float64_cols = [col for col in X.columns if X[col].dtype == 'float64']
if float64_cols:
    print(f"  • {len(float64_cols)} features using float64 could use float32")

print("\n" + "="*100)
print("\n✓ Data quality audit complete. Proceed with fixes in engineer.py")